## Computational Aspects of Complex Reflection Groups

Götz Pfeiffer - University of Galway

# 3. Cosets: Enumerating Complex Reflection Groups

![Numbers](images/numbers.jpg)

## Setup

First, reload the algorithms from earlier ...

In [ ]:
push!(LOAD_PATH, "./julia")
using permutation, orbits
using Graphs, GraphPlot
using variants, examples

## Complex Reflection Groups ...

... don't have
* a well-behaved length function
* simple reflections as generators
* root systems
* ...

But they do have
* a reflection representation
* parabolic subgroups
* ...
* a nice (Coxeter-like) presentation

Questions:
* how to construct the reflection representation systematically?
* how to turn the presentation into a usable group?

Here, we will only address the latter question, in the form of the **Todd-Coxeter coset enumeration** procedure,
a further variant of the orbit algorithm which potentially converts the group presentation into a finite permutation group. 

## Data Nodes

* Recall all the modified orbit algorithms.
* Here is all in one:  words, transversal and images.

In [ ]:
function orbit_and_more(aaa, x, under)
    list = [x]
    words = [[]]
    reps = [aaa[1]^0]
    images = [Int[] for a in aaa]
    i = 0
    while i < length(list)
        i += 1
        for (k, a) in enumerate(aaa)
            z = under(list[i], a)
            l = findfirst(==(z), list)
            if l == nothing
                push!(list, z)
                push!(words, onWords(words[i], k))
                push!(reps, reps[i] * a)
                l = length(list)
            end
            push!(images[k], l)
        end
    end
    return (list = list, words = words, reps = reps, images = images)
end;

In [ ]:
function orbit_and_more1(aaa, x, under)
    data = (list = [x], words = [[]], reps = [aaa[1]^0], images = [Int[] for a in aaa])
    i = 0
    while i < length(data.list)
        i += 1
        for (k, a) in enumerate(aaa)
            z = under(data.list[i], a)
            l = findfirst(==(z), data.list)
            if l == nothing
                push!(data.list, z)
                push!(data.words, onWords(data.words[i], k))
                push!(data.reps, data.reps[i] * a)
                l = length(data.list)
            end
            push!(data.images[k], l)
        end
    end
    return data
end;

* for example

In [ ]:
orb = orbit_and_more(transpositions(5), Set([1,2]), onSets)

In [ ]:
elist = union(enumerate.(orb.images)...)
graph = SimpleGraph(Edge.(filter(x -> !=(x...), elist)))
gplot(graph, nodelabel=vertices(graph))

* Managing all these property lists in parallel is a bit unwieldy, and perhaps confusing in the long run ...
* Let's transpose the setup and store properties as data with each node.

### Data Type

* An `Item` is a node object with **attributes**
  * `key`: an element of the domain that is acted upon
  * `idx`: its position in the list containing the orbit
  * `next`: a list of child nodes, one for each generator
  * `data`: other useful information

In [ ]:
struct Item
    key
    next::Array{Item}
    data::Dict{Symbol, Any}
    Item(key) = new(key, [], Dict())
end

### Constructor

* An `Item` object is constructed from a key $x \in X$.


In [ ]:
item = Item([1,2])

### Printing

* It will be convenient to install a method that prints a meaningful representation of an `Item` object.

In [ ]:
import Base: show
show(io::IO, item::Item) = print(io, "Item(", item.key, ")")

In [ ]:
item

###  Comparison

* Items are compared with respect to their keys, for equality, and for size.

In [ ]:
import Base: ==, isless
==(lft::Item, rgt::Item) = lft.key == rgt.key
isless(lft::Item, rgt::Item) = lft.key < rgt.key

In [ ]:
item == item, item < item, item <= item

In [ ]:
Set([item, item])

### Data Orbits

* We can now reformulate the above omnibus orbit algorithm in terms of such items.

In [ ]:
function orbit_with_data(aaa, item, under)
    list = [item]
    item.data[:idx] = 1  
    item.data[:rep] = aaa[1]^0
    item.data[:word] = []
    for x in list
        for (k, a) in enumerate(aaa)
            y = Item(under(x.key, a))
            l = findfirst(==(y), list)
            if l == nothing
                push!(list, y);
                y.data[:idx] = length(list)
                y.data[:rep] = x.data[:rep] * a
                y.data[:word] = onWords(x.data[:word], k)
                z = y
            else
                z = list[l]
            end
#            x.next[k] = z.data[:idx]
            push!(x.next, z)
        end
    end
    return list
end

In [ ]:
orb = orbit_with_data(transpositions(5), Item(Set([1,2])), onSets)

In [ ]:
[x.data for x in orb]

In [ ]:
[[y.data[:idx] for y in x.next] for x in orb]

In [ ]:
edge(x::Item, y::Item) = x.data[:idx], y.data[:idx]
elist = union([[edge(x, y) for y in x.next] for x in orb]...)

In [ ]:
graph = SimpleGraph(Edge.(filter(x -> !=(x...), elist)))
gplot(graph, nodelabel=vertices(graph))

* Permutations ...

In [ ]:
[Perm([x.next[i].data[:idx] for x in orb]) for i in eachindex(orb[1].next)]

## Example

* The complex reflection group $G_{12}$ has a presentation 
$$
\langle
s_1, s_2, s_3 \mid
s_1^2 = s_2^2 = s_3^2 = 1,\,
s_1 s_2 s_3 s_1 = s_2 s_3 s_1 s_2 = s_3 s_1 s_2 s_3
\rangle
$$
* Let's try and enumerate its elements systematically.

## Smart Nodes

* We will use a similar data structure, `Node`, for the purpose of coset enumeration.
* Here, the `idx` attribute is used to identify `Node` objects.
* And a `data` attribute is shared between all `Node` objects.

In [ ]:
struct Node
    idx::Int
    word::Vector{Int}
    flat::Vector{Node}
    next::Vector{Union{Node, Nothing}}
    data::Dict{Symbol, Any}
    function Node(word, data)
        l = length(data[:list])
        next = similar(data[:gens], Nothing)
        node = new(l + 1, word, [], next, data)
        push!(data[:list], node)
        data[:active] += 1
        return node
    end
end

### Print

In [ ]:
import Base: show
show(io::IO, node::Node) = print(io, "Node(", node.idx, ")")

In [ ]:
data = Dict(:list => [], :gens => [1,2,3], :active => 0)
node = Node([], data)

In [ ]:
node.next

### Comparison

In [ ]:
import Base: ==, isless
==(lft::Node, rgt::Node) = lft.idx == rgt.idx
isless(lft::Node, rgt::Node) = lft.idx < rgt.idx

In [ ]:
node == node, node < node, node <= node, node > node

## Coset Enumeration

* Q: What is $G = \langle S \mid R \rangle$?
* A: Todd-Coxeter!

* Suppose that a group $G$ is given by a **presentation** $\langle S \mid R \rangle$, consisting of a (finite) set $S$ of abstract **generators** $s_1, s_2, \dots, s_k$, and a (finite) list $R$ of **relations** $l_j = r_j$, where both $l_j$ and $r_j$ are words in $S \cup S^{-1}$.

* For convenience, we assume that $S$ is closed under inverses: $S  = S^{-1}$.

* We wish to enumerate the elements of $G$ (hoping that $G$ is a finite group), or more generally, the cosets of a subgroup $H$ of $G$ (hoping that $H$ has finite index in $G$).

* A priori, neither the domain $X$ being acted upon (by $G$), nor the edges of the action graph are known.

* Strategy: define new nodes as images of old nodes under a generator, but be prepared to identify this node with an existing one, if the relations imply they are the same.

* For this, each `Node` object $x$ has
  * a unique ID `idx` (where `idx` $ = n \iff x = x_n$),
  * a word `word` $ \in S^*$ (corresponding to a path in the BFS spanning tree of the action graph),
  * images $x$.`next`$[s] = x.s$ for each $s \in S$ (where $x.s \in X \cup \{ \perp \}$)
  * a reference $x$.`flat`$ \in X \cup \{ \perp \}$ to the node it has possibly been replaced by.
  
* Eventually, we want that $x.s \in X$ for all $x \in X$, $s \in S$.

###  Flatness

* A node $x$ is **active** if $x$.`flat`$ = {\perp}$.

In [ ]:
is_active(node::Node) = isempty(node.flat)

In [ ]:
is_active(node)

In [ ]:
push!(node.flat, node)
is_active(node)

In [ ]:
pop!(node.flat)
is_active(node)

* Each node $x \in X$ has an associated active node $x^{\flat}$ defined recursively as
$$
x^{\flat} = \begin{cases}
x, & \text{if } x.\text{flat} = {\perp}\\
(x.\text{flat})^{\flat}, & \text{else}
\end{cases}
$$

In [ ]:
flat(node::Node) = isempty(node.flat) ? node : flat(node.flat[1])

In [ ]:
flat(node)

### Images

* Recall that $S^{-1} = S$.  Assume that `data.invr` holds the map $s \mapsto s^{-1}$.
* In words, we write $-s$ for $s^{-1}$.  
* So to find $x.s$ for $s \in S = S^{-1}$ we need to replace $s$ by `data.invr`$[-s]$ first, if $s < 0$.

In [ ]:
function getImage(node::Node, s::Int)
    s < 0 ? node.next[node.data[:invr][-s]] : node.next[s]
end

??? define this as `node[s]`?

* To sprout a new node $x.s$:

In [ ]:
function sprout(node::Node, s::Int)
    next = Node(onWords(node.word, s), node.data)
    node.next[s] = next
    next.next[node.data[:invr][s]] = node
    return next
end

### Actions

* We will work with two distinct actions:
  * a **partial action** which returns `nothing` if an image does not exist (yet)
  * a **sprouting action** which sprouts a new node if necessary.

In [ ]:
function onNodesPartial(node::Node, s::Int)
    next = getImage(node, s)
    next == nothing ? next : flat(next)
end

In [ ]:
function onNodesSprout(node::Node, s::Int)
    next = getImage(node, s)
    next == nothing ? sprout(node, s) : flat(next)
end

* Both actions need only be defined on the generators $s \in S$, and can then be applied to words in $S^*$.

In [ ]:
function nodeUnderWordSprout(node::Node, word::Vector{Int})
    for s in word
        node = onNodesSprout(node, s)
    end
    return node
end

In [ ]:
function nodeUnderWordPartial(node::Node, word::Vector{Int})
    for s in word
        node = onNodesPartial(node, s)
        node == nothing && return node
    end
    return node
end

### Enumerate!

* We now formulate the `tabulate` procedure which takes a presentation `genrel` for a group $G$ as input and produces a permutation group as output.  Specifically, `genrel` has components
  * `gens`: a list `[1..n]` of abstract generators $S = S^{-1}$
  * `rels`: a list of relations expressed as pairs of word in $S$
  * `invr`: the map $S \to S: s \mapsto s^{-1}$
  * `sbgp`: a subset of $S$, generating a subgroup $H$ of $G$.

In [ ]:
function tabulate(genrel)

    # initialize.
    data = Dict(:list => [], :active => 0, :gens => genrel.gens, :invr => genrel.invr)
    data[:variants] = variantsRelations(genrel)
    node = Node([], data)
    
    # first close the subgroup tables.
    for word in genrel.sbgp
        trace(node, word)
    end
    
    # process nodes in the queue.
    for node in data[:list]
        for s in genrel.gens
            process(node, s)
        end
    end

    # return data.
    return data
end

###  Tracing Words

* To trace a node $x$ under a word $w$ means to make sure that $x.w = x$, using the sprouting action.
* If $w \in H$ then $x_1.w = x_1$ should hold.
* If $l = r$ is a relation then $w:= l/r = 1$ and $x.w = x$ should hold for any $x \in X$.
* In any case, for the last letter of $w$, we carefully check if the resulting coset is already known or not.

In [ ]:
function trace(node::Node, word::Vector{Int})
    other = nodeUnderWordSprout(node, word[1:end-1])
    updateEdge(other, word[end], node)
end

### Processing a Node under a Generator

* To find $x.s$, use variants of the relations to express $s$ as a word $w$ in the generators and check if $x.w$ is determined already.  If so, carefully set $x.s$ to $s.w$.  If this doesn't work out, create a new node $x.s$.

In [ ]:
function process(node::Node, s::Int)
    for variant in node.data[:variants][s]
        if is_active(node)
            next = nodeUnderWordPartial(node, variant)
            next == nothing || updateEdge(node, s, next)
        end
    end
    is_active(node) && node.next[s] == nothing && sprout(node, s)
end

### Edges

* In the (directed) .graph of a group action, an edge $x \stackrel{s}{\longrightarrow} y$ always comes with the opposite edge $y \stackrel{s^{-1}}{\longrightarrow} x$.
* Thus, carefully updating $x.s = y$ always refers to two edges of the graph.

In [ ]:
function updateEdge(node::Node, s::Int, next::Node)
    setImage(node, s, next)
    setImage(next, node.data[:invr][s], node)
end

* Carefully setting $x.s$ to $y$ means 
  * checking if $x.s$ is already defined; if not, set $x.s$ to $y$.
  * Otherwise, with $x.s = z$, say, check if $y = z$:  if so there is nothing to do.
  * Otherwise, set $z = y$ (or $y = z$ depending on which came first) and live with the consequenses ...

In [ ]:
function setImage(node::Node, s::Int, next::Node)
    if node.next[s] != nothing
        pair = sort(flat.([next, node.next[s]]))
        ==(pair...) || mergeNodes(pair[2], pair[1])  # coincidence: stack!
    else
        node.next[s] = next           # deduction!
    end
end

* to merge nodes $z$ and $y$:
  * set $z$.`flat` to $y$
  * for each $z.s \neq {\perp}$, carefully update $z.s = y.s$.

In [ ]:
function mergeNodes(node::Node, other::Node)
    push!(node.flat, other)
    node.data[:active] -= 1
    for (s, next) in enumerate(node.next) 
        next == nothing || updateEdge(other, s, next)
    end
end

In [ ]:
G = examples.G12

In [ ]:
data = tabulate(G)

In [ ]:
nodes = filter(is_active, data[:list])
println(nodes)

In [ ]:
println([flat(x.next[1]).idx for x in nodes])

In [ ]:
cycles(Perm(sortperm([flat(x.next[1]).idx for x in nodes]))^-1) #;-)

In [ ]:
gens = [Perm(sortperm([flat(x.next[s]).idx for x in nodes]))^-1 for s in G.gens]

In [ ]:
using permgroup
sizeOfGroup(PermGp(gens, gens[1]^0))

* Next: carefully extract and plot the graph (on active nodes only)!

In [ ]:
elist = union([[(n.idx, flat(x).idx) for x in n.next] for n in nodes]...)

In [ ]:
graph = SimpleGraph(Edge.(filter(x -> !=(x...), elist)))
gplot(graph, nodelabel=vertices(graph))

##  Exercises, etc.

* ($*$) Modify the coset enumeration procedure so that it applies to monoid presentations.

* Find matrices for the reflection representation of a given complex reflection group $G$.

* Find a way to enumerate the (conjugacy classes of) parabolic subgroups $P$ of a complex reflection group $G$.

* Compute the normalizer of parabolic subgroup $P$ in $G$.  Does $P$ always have a complement $H$? If so, does $H$ have a natural set of generators? 

* ...